In [203]:
from libraries import *
from parameters import *

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

In [234]:
os.getcwd()
os.chdir(projectDir)

In [235]:
adata = sc.read_h5ad('./DATA/sc_training.h5ad')
myDF_2 = pd.crosstab(adata.obs.condition,adata.obs.state)
Y = myDF_2.div(myDF_2.sum(axis=1), axis=0)

In [236]:
Y

state,cycling,effector,other,progenitor,terminal exhausted
condition,,,,,
Arid4b,0.426101,0.117925,0.011006,0.056604,0.388365
Arid5b,0.513739,0.044546,0.017485,0.011241,0.412989
Atf2,0.297872,0.248227,0.028369,0.120567,0.304965
Batf,0.000000,0.000000,0.166667,0.666667,0.166667
Crem,0.308804,0.425756,0.017083,0.034166,0.214192
...,...,...,...,...,...
Tpt1,0.280000,0.160000,0.000000,0.440000,0.120000
Unperturbed,0.392125,0.209723,0.017276,0.067497,0.313379
Yy1,0.120000,0.200000,0.000000,0.600000,0.080000


In [237]:
allReg = pd.read_csv("./DATA/AllRegResults.csv")
allReg = allReg[~allReg.guides.isin(['cycling', 'effector', 'other', 'exhausted'],)]
allReg.pval = allReg.pval.astype(float)

/tmp/ipykernel_40101/2071413347.py:1: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  allReg = pd.read_csv("./DATA/AllRegResults.csv")


In [238]:
coefs = pd.pivot(allReg[['guides','coef', 'respGene']],index='guides', columns='respGene', values='coef')
pvals = pd.pivot(allReg[['guides','pval', 'respGene']],index='guides', columns='respGene', values='pval')
#coefs[pvals > 0.05] = 0 

coefs = coefs.loc[coefs.index.isin(coefs.columns),]
coefs = coefs.loc[:,coefs.index].T

In [239]:
coefs

guides,Arid4b,Arid5b,Atf2,Batf,Crem,Ctnnb1,Dkk3,Dvl1,Dvl2,Dvl3,...,Sub1,Tbx21,Tcf3,Tcf7,Tox,Tox2,Tpt1,Yy1,Zeb2,Zfp292
guides,,,,,,,,,,,,,,,,,,,,,
Arid4b,-0.0850,-0.0112,0.0525,-0.1300,0.0086,0.0340,-0.0483,-0.0093,-0.0223,-0.0307,...,-0.0079,0.4273,0.0283,0.0156,0.1179,-0.0143,-0.0970,-0.0540,-0.0049,0.0236
Arid5b,-0.0306,-0.0901,0.0223,-0.1003,-0.0239,-0.0933,0.0239,-0.0007,-0.0189,0.0141,...,-0.0013,0.0406,-0.0120,-0.0351,-0.0433,-0.0242,0.0711,-0.0198,-0.0366,0.0230
Atf2,-0.0045,-0.0114,-0.0330,-0.0539,0.0019,-0.0472,0.0176,0.0071,-0.0067,0.0070,...,-0.0002,0.0105,0.0162,-0.0081,0.0058,0.0010,0.0704,0.0209,-0.0029,0.0152
Batf,0.0806,-0.0506,-0.0143,-0.0202,-0.0361,-0.1007,-0.0002,-0.0140,0.0099,-0.0474,...,-0.0728,-0.0702,0.0260,-0.0192,-0.0302,0.0214,0.0240,0.0133,0.0345,-0.0083
Crem,-0.1027,-0.1138,-0.0210,-0.1084,-0.0236,0.0167,-0.0302,0.0384,-0.0500,-0.0591,...,-0.0341,-0.0652,-0.0086,0.0199,-0.1937,-0.0662,-0.0713,0.1486,-0.0168,-0.0217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Tox2,0.0022,-0.0021,-0.0043,-0.0033,-0.0017,-0.0049,-0.0042,-0.0038,-0.0008,-0.0002,...,-0.0041,-0.0049,-0.0041,-0.0022,-0.0052,-0.0026,-0.0046,-0.0049,-0.0035,-0.0033
Tpt1,-0.0396,-0.0364,0.0138,-0.5446,0.0380,0.0430,-0.0507,-0.0139,-0.0344,-0.0283,...,0.0080,0.2572,-0.0050,0.0025,-0.0451,-0.0621,-1.0627,0.0741,0.0313,0.0950
Yy1,0.0036,0.0142,0.0109,-0.1121,-0.0016,-0.0457,-0.0332,-0.0138,-0.0095,0.0002,...,0.0478,-0.0417,-0.0101,0.0124,-0.0133,0.0126,0.0243,-0.0631,-0.0047,0.0081


In [240]:
Y = Y.loc[coefs.index,:]

In [241]:
Y

state,cycling,effector,other,progenitor,terminal exhausted
guides,,,,,
Arid4b,0.426101,0.117925,0.011006,0.056604,0.388365
Arid5b,0.513739,0.044546,0.017485,0.011241,0.412989
Atf2,0.297872,0.248227,0.028369,0.120567,0.304965
Batf,0.000000,0.000000,0.166667,0.666667,0.166667
Crem,0.308804,0.425756,0.017083,0.034166,0.214192
...,...,...,...,...,...
Tox2,0.530810,0.057004,0.027002,0.017309,0.367874
Tpt1,0.280000,0.160000,0.000000,0.440000,0.120000
Yy1,0.120000,0.200000,0.000000,0.600000,0.080000


In [242]:
#model = DecisionTreeRegressor()
#model = LinearRegression()
model = KNeighborsRegressor(n_neighbors=5)
#model = RandomForestRegressor(n_estimators=40, max_depth=10, random_state=2)

# define the evaluation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, X, Y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force the scores to be positive
n_scores = absolute(n_scores)
# summarize performance
print('MAE: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

MAE: 0.111 (0.035)


In [243]:
model.fit(X,Y)

KNeighborsRegressor()

In [246]:
Y

state,cycling,effector,other,progenitor,terminal exhausted
guides,,,,,
Arid4b,0.426101,0.117925,0.011006,0.056604,0.388365
Arid5b,0.513739,0.044546,0.017485,0.011241,0.412989
Atf2,0.297872,0.248227,0.028369,0.120567,0.304965
Batf,0.000000,0.000000,0.166667,0.666667,0.166667
Crem,0.308804,0.425756,0.017083,0.034166,0.214192
...,...,...,...,...,...
Tox2,0.530810,0.057004,0.027002,0.017309,0.367874
Tpt1,0.280000,0.160000,0.000000,0.440000,0.120000
Yy1,0.120000,0.200000,0.000000,0.600000,0.080000


In [244]:
model.predict(X)

array([[0.27541754, 0.16183774, 0.01250653, 0.31518571, 0.23505247],
       [0.43575537, 0.16309706, 0.01716922, 0.07477485, 0.30920349],
       [0.34364591, 0.26163149, 0.01400296, 0.1260187 , 0.25470094],
       [0.052     , 0.056     , 0.03533333, 0.78733333, 0.06933333],
       [0.32734301, 0.29904671, 0.02179558, 0.1549883 , 0.19682641],
       [0.3164224 , 0.24026918, 0.00729158, 0.20723918, 0.22877765],
       [0.37231913, 0.2091872 , 0.00552531, 0.15348589, 0.25948247],
       [0.40551027, 0.17293068, 0.01524802, 0.11225138, 0.29405965],
       [0.37739977, 0.24399427, 0.0108208 , 0.1218083 , 0.24597685],
       [0.30166183, 0.18372382, 0.01380236, 0.26228742, 0.23852456],
       [0.26789089, 0.12159579, 0.00519759, 0.39301439, 0.21230133],
       [0.27266822, 0.25008778, 0.005532  , 0.27194575, 0.19976626],
       [0.20709744, 0.3004476 , 0.00560506, 0.31386721, 0.1729827 ],
       [0.197388  , 0.20967949, 0.013799  , 0.41652446, 0.16260905],
       [0.26460516, 0.18373957, 0.

In [98]:
X_test = 

,cycling,effector,other,progenitor,terminal exhausted
Eef2,2.339576,2.37774,2.244495,2.449251,2.353572
Ep300,0.130523,0.14833,0.253649,0.145159,0.120915
Klf2,0.065644,0.049613,0.037668,0.116205,0.041549
Rps6,2.422122,2.220766,2.443958,2.530816,2.468894
Runx3,0.507188,0.433834,0.687709,0.337062,0.506031
...,...,...,...,...,...
Sox4,0.000305,0.0,0.020719,0.003679,0.0
Arid5b,0.24081,0.32644,0.209493,0.28265,0.233735
Tox2,0.002197,0.006353,0.003321,0.001873,0.004833
Nr3c1,0.245159,0.312605,0.271575,0.209807,0.27285
